In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LazySegmentTree:
  def __init__(self, data):
    self.size = 1 << (len(data) - 1).bit_length()
    self.L = [0] * (2 * self.size)
    self.lazy = [0] * (2 * self.size)
    self.L[self.size:self.size + len(data)] = data
    for i in reversed(range(self.size)):
      self.L[i] = self.L[i*2] + self.L[i*2+1]

  def push(self, i, s, e) :
    if not self.lazy[i] : return

    if i < self.size :
      self.lazy[i*2] += self.lazy[i]
      self.lazy[i*2+1] += self.lazy[i]
    elif self.lazy[i] & 1: #리프노드이면서 lazy[i]가 홀수면
      self.L[i] ^= 1 #반전
    self.lazy[i] = 0

  def query(self, l, r):
    return self._query(l, r, 1, 0, self.size)

  def _query(self, l, r, i, nl, nr) :
    self.push(i, nl, nr)
    
    if r <= nl or nr <= l : return 0
    if l <= nl and nr <= r : return self.L[i]
    mid = (nl + nr) // 2
    return self._query(l, r, i*2, nl, mid) + self._query(l, r, i*2+1, mid, nr)
  
  def add(self, l, r, x):
    self._add(l, r, x, 1, 0, self.size)

  def _add(self, l, r, x, i, nl, nr) :
    self.push(i, nl, nr)

    if r <= nl or nr <= l : return
    if l <= nl and nr <= r :
      self.lazy[i] += x
      self.push(i, nl, nr)
      return
  
    mid = (nl + nr) // 2
    self._add(l, r, x, i*2, nl, mid)
    self._add(l, r, x, i*2+1, mid, nr)
    self.L[i] = self.L[i*2] + self.L[i*2+1]

def sol() :
  N, M = map(int, input().split())
  lst = LazySegmentTree([0] * (N + 2))

  ans = []
  for _ in range(M) :
    q, *l = map(int, input().split())
    if q == 0 :
      s, e = l
      lst.add(s, e+1, 1)
    elif q == 1 :
      s, e = l
      ans.append(lst.query(s, e+1))
      print(lst.L)
  
  sys.stdout.write('\n'.join(map(str, ans)))

sol()

### 아이디어
- 켜진 스위치의 개수를 세야 하므로 누적합 세그트리를 만들어야 한다.
- update마다 변경량을 1씩 더해주는 것으로 구현하려 했는데, 리프노드를 제외한 곳의 처리를 어떻게 해야할지 감이 안온다. 잘못된 접근같다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LazySegmentTree:
  def __init__(self, data):
    self.size = 1 << (len(data) - 1).bit_length()
    self.L = [0] * (2 * self.size)
    self.lazy = [0] * (2 * self.size)
    self.L[self.size:self.size + len(data)] = data
    for i in reversed(range(self.size)):
      self.L[i] = self.L[i*2] + self.L[i*2+1]

  def push(self, i, s, e) :
    if not self.lazy[i] : return

    if i < self.size :
      self.lazy[i*2] ^= 1
      self.lazy[i*2+1] ^= 1
      
      res = 0
      if self.lazy[i*2] : #왼쪽 자식 전체가 반전될 예정이면
        res += (e - s) // 2 - self.L[i*2] #구간의 길이에 대해 자식들의 켜진 개수를 반전
      else :
        res += self.L[i*2] #그렇지 않을 경우 자식의 값을 그대로 가져옴
      
      if self.lazy[i*2+1] :
        res += (e - s) // 2 - self.L[i*2+1]
      else :
        res += self.L[i*2+1]
      
      self.L[i] = res

    else:
      self.L[i] ^= 1 #쿼리 처리
    self.lazy[i] = 0

  def query(self, l, r):
    return self._query(l, r, 1, 0, self.size)

  def _query(self, l, r, i, nl, nr) :
    self.push(i, nl, nr)
    
    if r <= nl or nr <= l : return 0
    if l <= nl and nr <= r : return self.L[i]
    mid = (nl + nr) // 2
    return self._query(l, r, i*2, nl, mid) + self._query(l, r, i*2+1, mid, nr)
  
  def add(self, l, r, x):
    self._add(l, r, x, 1, 0, self.size)

  def _add(self, l, r, x, i, nl, nr) :
    self.push(i, nl, nr)

    if r <= nl or nr <= l : return
    if l <= nl and nr <= r :
      self.lazy[i] += x
      self.push(i, nl, nr)
      return
  
    mid = (nl + nr) // 2
    self._add(l, r, x, i*2, nl, mid)
    self._add(l, r, x, i*2+1, mid, nr)
    self.L[i] = self.L[i*2] + self.L[i*2+1]

def sol() :
  N, M = map(int, input().split())
  lst = LazySegmentTree([0] * (N + 2))

  ans = []
  for _ in range(M) :
    q, *l = map(int, input().split())
    if q == 0 :
      s, e = l
      lst.add(s, e+1, 1)
    elif q == 1 :
      s, e = l
      ans.append(lst.query(s, e+1))
  
  sys.stdout.write('\n'.join(map(str, ans)))

sol()

### 풀이
- propagation할 때 가능한 만큼 쿼리를 처리한다는 발상과 XOR 연산의 특징을 이용한다.


In [ ]:
import io, os, sys
from operator import xor, add
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LazySegmentTree:
  def __init__(self, data, f=lambda x, y: x+y, fl=lambda i, x, cnt: i+x*cnt, fll=lambda x, y: x+y, default=0, ldefault=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.default = default
    self.ldefault = ldefault
    self.len = len(data)
    self.L = [default] * self.len + data
    self.lazy = [ldefault] * self.len
    self.cnt = [0] * self.len + [1] * self.len

    for i in reversed(range(self.len)):
      self.L[i] = self.f(self.L[i*2], self.L[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _apply(self, i) :
    if self.lazy[i] == self.ldefault: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.ldefault 

  def _propagate(self, i):
    for h in reversed(range(1, self.len.bit_length()+1)) :
      idx = i >> h

      if idx == self.default : continue
      self._apply(idx)

  def _push(self, idx, val) :
    self.L[idx] = self.fl(self.L[idx], val, self.cnt[idx])
    if idx < self.len :
      self.lazy[idx] = self.fll(self.lazy[idx], val)

  def _build(self, idx) :
    while idx :
      if idx < self.len :
        self._apply(idx)
        self.L[idx] = self.f(self.L[idx*2], self.L[idx*2+1])
      idx >>= 1

  def query(self, _s, _e) : #[s, e]
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    res = self.default
    while s < e :
      if s & 1 :
        res = self.f(res, self.L[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.L[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, _s, _e, x):
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e)

def sol() :
  N, M = map(int, input().split())
  lst = LazySegmentTree([0] * N, f=add, fl=lambda i, x, cnt: cnt-i if x else i ,fll=xor)

  ans = []
  for _ in range(M) :
    q, *l = map(int, input().split())
    if q == 0 :
      s, e = l
      lst.update(s-1, e-1, 1)
    elif q == 1 :
      s, e = l
      ans.append(lst.query(s-1, e-1))
      print(lst.L)
  
  sys.stdout.write('\n'.join(map(str, ans)))

sol()

- 구간 내의 켜져있는 스위치의 합을 구할 것이므로, f는 add가 된다.
- lazy는 bool, fll은 xor으로, 구간 내의 스위치 상태를 바꿀지의 여부를 저장한다.
- 구간내에 스위치의 상태가 반전된다면, 쿼리 적용시 켜진 스위치의 개수는 (구간의 길이) - (켜져있었던 스위치의 개수), 즉 개수가 길이만큼 반전된다.